# Load adapters with 🤗 PEFT

- https://huggingface.co/docs/transformers/en/peft

## Load a PEFT adapter

In [59]:
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "ybelkada/opt-350m-lora"
model = AutoModelForCausalLM.from_pretrained(peft_model_id)

In [60]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "facebook/opt-350m"
peft_model_id = "ybelkada/opt-350m-lora"

model = AutoModelForCausalLM.from_pretrained(model_id)
model.load_adapter(peft_model_id)

## Add a new adapter

In [61]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import LoraConfig

model_id = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_id)

lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights=False
)

model.add_adapter(lora_config, adapter_name="adapter_1")

In [62]:
# attach new adapter with same config
model.add_adapter(lora_config, adapter_name="adapter_2")

In [63]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
text = "Hello"
inputs = tokenizer(text, return_tensors="pt")

In [64]:
# use adapter_1
model.set_adapter("adapter_1")
output_disabled = model.generate(**inputs)
print(tokenizer.decode(output_disabled[0], skip_special_tokens=True))

# use adapter_2
model.set_adapter("adapter_2")
output_enabled = model.generate(**inputs)
print(tokenizer.decode(output_enabled[0], skip_special_tokens=True))

/Users/koyonakayama/workspace/hugging_face_tutorial/.env/lib/python3.9/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Hello, I'm a newbie to this sub. I'm looking for a good place to
Hello, I'm a newbie to this sub. I'm looking for a good place to


## Enable and disable adapters

In [65]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import PeftConfig

model_id = "facebook/opt-350m"
adapter_model_id = "ybelkada/opt-350m-lora"
tokenizer = AutoTokenizer.from_pretrained(model_id)
text = "Hello"
inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForCausalLM.from_pretrained(model_id)
peft_config = PeftConfig.from_pretrained(adapter_model_id)

# to initiate with random weights
peft_config.init_lora_weights = False

model.add_adapter(peft_config)
model.enable_adapters()
output = model.generate(**inputs)

In [66]:
tokenizer.decode(output[0], skip_special_tokens=True)

'Hello------------------'

In [67]:
model.disable_adapters()
output = model.generate(**inputs)

In [68]:
tokenizer.decode(output[0], skip_special_tokens=True)

"Hello, I'm a newbie to this sub. I'm looking for a good place to"

## Train a PEFT adapter

In [97]:
from datasets import load_dataset

eli5 = load_dataset("eli5_category", split="train[:100]")

In [98]:
eli5 = eli5.train_test_split(test_size=0.2)

In [99]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

In [100]:
eli5 = eli5.flatten()

In [101]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [102]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

In [103]:
block_size = 128

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [104]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

In [105]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [106]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

In [107]:
training_args = TrainingArguments(
    output_dir="peft",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [108]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [109]:
model.add_adapter(peft_config)

In [110]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)
trainer.train()

 33%|███▎      | 33/99 [00:26<00:43,  1.51it/s]





                                               

 33%|███▎      | 33/99 [00:28<00:43,  1.51it/s]



{'eval_loss': 4.105483531951904, 'eval_runtime': 1.306, 'eval_samples_per_second': 27.565, 'eval_steps_per_second': 3.829, 'epoch': 1.0}


 67%|██████▋   | 66/99 [00:52<00:21,  1.56it/s]





                                               

 67%|██████▋   | 66/99 [00:54<00:21,  1.56it/s]



{'eval_loss': 4.1044769287109375, 'eval_runtime': 1.3089, 'eval_samples_per_second': 27.504, 'eval_steps_per_second': 3.82, 'epoch': 2.0}


100%|██████████| 99/99 [01:18<00:00,  1.55it/s]





                                               

100%|██████████| 99/99 [01:19<00:00,  1.55it/s]


100%|██████████| 99/99 [01:19<00:00,  1.24it/s]

{'eval_loss': 4.104079723358154, 'eval_runtime': 1.3048, 'eval_samples_per_second': 27.591, 'eval_steps_per_second': 3.832, 'epoch': 3.0}
{'train_runtime': 79.7955, 'train_samples_per_second': 9.775, 'train_steps_per_second': 1.241, 'train_loss': 4.1099341806739265, 'epoch': 3.0}


TrainOutput(global_step=99, training_loss=4.1099341806739265, metrics={'train_runtime': 79.7955, 'train_samples_per_second': 9.775, 'train_steps_per_second': 1.241, 'total_flos': 26183089520640.0, 'train_loss': 4.1099341806739265, 'epoch': 3.0})

In [111]:
save_dir="peft"

In [112]:
model.save_pretrained(save_dir)
model = AutoModelForCausalLM.from_pretrained(save_dir)

/Users/koyonakayama/workspace/hugging_face_tutorial/.env/lib/python3.9/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


## Add additional trainable layers to a PEFT adapter

In [113]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import LoraConfig

model_id = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_id)

lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    modules_to_save=["lm_head"],
)

model.add_adapter(lora_config)